In [ ]:
# Standard libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import IntProgress
from IPython.display import display

# Append base directory
import os,sys,inspect
rootname = "pub-2020-exploratory-analysis"
# thispath = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
thispath = os.getcwd()
rootpath = os.path.join(thispath[:thispath.index(rootname)], rootname)
sys.path.append(rootpath)
print("Appended root directory", rootpath)

from mesostat.metric.metric import MetricCalculator
from mesostat.utils.qt_helper import gui_fname, gui_fnames, gui_fpath
from mesostat.utils.hdf5_io import DataStorage
from mesostat.utils.pandas_helper import pd_query

from lib.gallerosalas.data_fc_db_raw import DataFCDatabase
import lib.analysis.triplet_analysis.mousewise as mousewise
import lib.analysis.triplet_analysis.calc_reader_mousewise as calc_reader

%load_ext autoreload
%autoreload 2

In [ ]:
params = {}
# params['root_path_data']  = gui_fpath("Path to data collection",  './')
params['root_path_data'] = '/media/alyosha/Data/TE_data/yasirdata_raw/'
# params['root_path_data'] = '/home/alyosha/data/yasirdata_raw/'
# params['root_path_data'] = '/media/aleksejs/DataHDD/work/data/yasir/yasirdata_raw'

In [ ]:
dataDB = DataFCDatabase(params)

In [ ]:
dataDB.calc_shortest_distances()

In [ ]:
print('mice', dataDB.mice)
print('nSessions', len(dataDB.sessions))
print('datatypes', dataDB.get_data_types())
print('nChannel', dataDB.get_nchannels('mou_5'))

In [ ]:
h5outname = 'gallerosalas_result_higher_order_df.h5'

In [ ]:
mc = MetricCalculator(serial=True, verbose=False, nCore=4)

In [ ]:
dataDB.channelAreasDF

## All - Distribution - Nosession

In [ ]:
prefixPath = '/media/alyosha/Data/TE_data/calculations/pr2/yasir-tex/'

pwdH5data = prefixPath + 'pr2_gallerosalas_multimouse_data.h5'
pwdH5rand = prefixPath + 'pr2_gallerosalas_multimouse_rand.h5'
# pwdH5data = prefixPath + 'pid_gallerosalas_multimouse_nbin_2_data.h5'
# pwdH5rand = prefixPath + 'pid_gallerosalas_multimouse_nbin_2_rand.h5'

In [ ]:
intervNames = dataDB.get_interval_names()
#trialTypes = ['Hit', 'CR', 'Miss', 'FA']
trialTypes = ['Hit', 'CR']

In [ ]:
dfSummary = calc_reader.summary_df(pwdH5data)
dfSummaryDataSizes = calc_reader.summary_update_data_sizes(dfSummary, dataDB)

In [ ]:
dfSummaryRand = calc_reader.summary_df(pwdH5rand)
dfSummaryRandSizes = calc_reader.summary_update_data_sizes(dfSummaryRand, dataDB)

### 1. Global Significance Testing

1. For all sweep get data size
2. Inside test, generate AdversaryDist for each data size, save as file
3. Load file as dict, provide as argument to test
4. In test, produce dataframe: Sweep -> (frac>Shuffle, pValSuffle, frac>Conserv, pValConserv)
5. Plot table

In [ ]:
pwdAdversarial = '/media/alyosha/Data/TE_data/pr2_rand_dist.h5'

In [ ]:
dictAdversarial = calc_reader.read_adversarial_distr_file(pwdAdversarial)

In [ ]:
rezTestShuffle = mousewise.plot_violin_test(pwdH5data, pwdH5rand, dfSummary, dfSummaryRand, thrBig=0.01)

In [ ]:
rezTestAdversarial = mousewise.plot_violin_test_adversarial(pwdH5data, dictAdversarial, dfSummaryDataSizes, thrBig=0.01)

In [ ]:
from mesostat.visualization.mpl_matrix import plot_df_2D_outer_product

In [ ]:
def inf_update(df, key, val):
    df.loc[df[key] == np.inf, key] = val
    df.loc[df[key] == -np.inf, key] = val
    return df

rezTestShuffle = inf_update(rezTestShuffle, '-log10(pval)', 1000)
rezTestAdversarial = inf_update(rezTestAdversarial, '-log10(pval)', 1000)

# Plot significance
print('Shuffle-Sign')
fig, ax = plt.subplots(ncols=3, figsize=(12,4), tight_layout=True)
for iPid, pidType in enumerate(['red', 'unique', 'syn']):
    dfQuery = pd_query(rezTestShuffle, {'atom': pidType})
    plot_df_2D_outer_product(ax[iPid], dfQuery, ['datatype', 'mousename'], ['trialType', 'intervName'],
                             '-log10(pval)', vmin=0, vmax=20, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']})
    
plt.savefig('pr2_tex_neglogpval_shuffle_outer2d.svg')
plt.show()


# Plot significance
print('Adv-Sign')
fig, ax = plt.subplots(ncols=3, figsize=(12,4), tight_layout=True)
for iPid, pidType in enumerate(['red', 'unique']):
    dfQuery = pd_query(rezTestAdversarial, {'atom': pidType})
    plot_df_2D_outer_product(ax[iPid], dfQuery, ['datatype', 'mousename'], ['trialType', 'intervName'],
                             '-log10(pval)', vmin=0, vmax=20, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']})
    
plt.savefig('pr2_tex_neglogpval_adversarial_outer2d.svg')
plt.show()

In [ ]:
print('Shuffle-Sign')
fig, ax = plt.subplots(ncols=3, figsize=(12,4), tight_layout=True)
for iPid, pidType in enumerate(['red', 'unique', 'syn']):
    dfQuery = pd_query(rezTestShuffle, {'atom': pidType})
    plot_df_2D_outer_product(ax[iPid], dfQuery, ['datatype', 'mousename'], ['trialType', 'intervName'],
                             'fracSign', vmin=0, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']})
    
plt.savefig('pr2_tex_significant_shuffle_outer2d.svg')
plt.show()


print('Adv-Sign')
fig, ax = plt.subplots(ncols=3, figsize=(12,4), tight_layout=True)
for iPid, pidType in enumerate(['red', 'unique']):
    dfQuery = pd_query(rezTestAdversarial, {'atom': pidType})
    plot_df_2D_outer_product(ax[iPid], dfQuery, ['datatype', 'mousename'], ['trialType', 'intervName'],
                             'fracSign', vmin=0, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']})
    
plt.savefig('pr2_tex_significant_adversarial_outer2d.svg')
plt.show()


print('Shuffle-Big')
fig, ax = plt.subplots(ncols=3, figsize=(12,4), tight_layout=True)
for iPid, pidType in enumerate(['red', 'unique', 'syn']):
    dfQuery = pd_query(rezTestShuffle, {'atom': pidType})
    plot_df_2D_outer_product(ax[iPid], dfQuery, ['datatype', 'mousename'], ['trialType', 'intervName'],
                             'fracBig', vmin=0, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']})
    
plt.savefig('pr2_tex_big_outer2d.svg')
plt.show()

In [ ]:
import h5py
testingThresholdsShu = {'unique': {}, 'red': {}, 'syn': {}}

with h5py.File(pwdH5rand, 'r') as h5f:
    for idx, row in dfSummaryRandSizes.iterrows():
        thrU1 = np.quantile(h5f[row['key']][:, 0], 0.99)
        thrU2 = np.quantile(h5f[row['key']][:, 0], 0.99)
        thrR = np.quantile(h5f[row['key']][:, 0], 0.99)
        thrS = np.quantile(h5f[row['key']][:, 0], 0.99)
        thrU = (thrU1 + thrU2) / 2
        rezDict = {'unique': thrU, 'red': thrR, 'syn': thrS}
        
        for k, v in rezDict.items():
            if row['nData'] in testingThresholdsShu[k].keys():
                testingThresholdsShu[k][row['nData']] += [v]
            else:
                testingThresholdsShu[k][row['nData']] = [v]

testingThresholdsShu = {pidType : {k: np.mean(v) for k,v in subDict.items()} for pidType, subDict in testingThresholdsShu.items()}

In [ ]:
# Testing thresholds 
testingThresholdsAdv = {pidType : {k: np.quantile(v, 0.99) for k,v in subDict.items()} for pidType, subDict in dictAdversarial.items()}

In [ ]:
testingThresholds = {
    'unique': testingThresholdsAdv['unique'],
    'red': testingThresholdsAdv['red'],
    'syn': testingThresholdsShu['syn']
}

In [ ]:
mousewise.plot_cdf(pwdH5data, dfSummaryDataSizes, testingThresholds=testingThresholds, printSummary=True)

## Mouse-average

In [ ]:
mousewise.barplot_avg(dataDB, pwdH5data, dfSummary, 'intervName', intervNames)

In [ ]:
mousewise.barplot_avg(dataDB, pwdH5data, dfSummary, 'trialType', trialTypes)

## Singlets

In [ ]:
mousewise.plot_top_singlets(dataDB, pwdH5data, dfSummary)

In [ ]:
dfTop1Dmean = mousewise.plot_top_singlets_bymouse_outer2D(dataDB, pwdH5data, dfSummary, 'syn',
                                                          func=np.nanmean, dropna=True, magThr=0.02)

In [ ]:
dfTop1Dmax = mousewise.plot_top_singlets_bymouse_outer2D(dataDB, pwdH5data, dfSummary, 'syn',
                                                          func=np.nanmax, dropna=True, magThr=0.02)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(20, 10), tight_layout=True)

for i, datatype in enumerate(['bn_trial', 'bn_session']):
    dfQuery = pd_query(dfTop1Dmean, {'datatype': datatype})
    dfQuery = dfQuery[dfQuery['trialType'] != 'None']
    ax[i].set_title(datatype)
    plot_df_2D_outer_product(ax[i], dfQuery, ['label'], ['datatype', 'mousename', 'intervName', 'trialType'],
                             'syn', vmin=0, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']}, 
                             dropEmpty=True)
    
#plt.savefig('pid_tex_big_outer2d.svg')
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(20, 10), tight_layout=True)

for i, datatype in enumerate(['bn_trial', 'bn_session']):
    dfQuery = pd_query(dfTop1Dmax, {'datatype': datatype})
    dfQuery = dfQuery[dfQuery['trialType'] != 'None']
    ax[i].set_title(datatype)
    plot_df_2D_outer_product(ax[i], dfQuery, ['label'], ['datatype', 'mousename', 'intervName', 'trialType'],
                             'syn', vmin=0, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']}, 
                             dropEmpty=True)
    
# plt.savefig('pid_tex_big_outer2d.svg')
plt.show()

In [ ]:
mousewise.plot_singlets_brainplot(dataDB, pwdH5data, dfSummary, 'intervName', intervNames)

In [ ]:
mousewise.plot_singlets_brainplot(dataDB, pwdH5data, dfSummary, 'trialType',
                                  trialTypes)

In [ ]:
mousewise.plot_singlets_brainplot_mousephase_subpre(dataDB, pwdH5data, dfSummary)

In [ ]:
mousewise.plot_singlets_brainplot_mousephase_submouse(dataDB, pwdH5data, dfSummary)

## Triplets

In [ ]:
mousewise.plot_top_triplets(dataDB, pwdH5data, dfSummary, nTop=20)#, dropChannels=['BC'])

In [ ]:
dfSelectedTriplets = mousewise.plot_filter_top_triplets_bymouse(dataDB, pwdH5data, dfSummary,
                                                                nTop=30, thrBig=0.01, nConsistent=4)

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(20, 10), tight_layout=True)

for i, datatype in enumerate(['bn_trial', 'bn_session']):
    ax[i].set_title(datatype)
    plot_df_2D_outer_product(ax[i], dfSelectedTriplets[datatype], ['label'],
                             ['mousename', 'intervName', 'trialType'],
                             'syn', vmin=0, vmax=0.1, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']}, 
                             dropEmpty=True)
    
plt.savefig('pr2_tex_big_triplets_outer2d.svg')
plt.show()

In [ ]:
def plot_triplet_clusters(fig, ax, dataDB, labelsS1, labelsS2, labelsTrg):
    labelsCanon = list(dataDB.map_channel_labels_canon().values())
    
    clusterDict = {
        'source1': [labelsCanon.index(l) for l in labelsS1],
        'source2': [labelsCanon.index(l) for l in labelsS2],
        'target': [labelsCanon.index(l) for l in labelsTrg],
    }
    
    dataDB.plot_area_clusters(fig, ax, clusterDict, haveLegend=True)

In [ ]:
clusterDictDict = {
    'bn_trial': {
        'rez1':  [['MOp', 'MOs', 'SSp-ll', 'SSp-tr', 'VISa', 'VISam'],  ['RSPd', 'RSPagl'], ['VISpl', 'VISpor', 'TEa']],
        'rez2':  [['AUDd', 'SSs'], ['SSp-n', 'SSp-m', 'SSp-bfd', 'VISal'], ['SSp-ll']],
        'rez3':  [['MOp', 'SSp-ul'], ['SSp-ll'], ['VISpm']],
        'rez4':  [['SSp-n'], ['RSPd'], ['TEa']]
    },
    'bn_session': {
        'rez1':  [['AUDpo', 'TEa'], ['VISp'], ['SSp-ll']],
        'rez2':  [['MOp', 'SSp-tr', 'SSp-ul'], ['SSp-ll', 'VISa'], ['VISpl', 'VISpor', 'VISli']],
        'rez3':  [['MOs'], ['RSPd'], ['MOp', 'RSPagl', 'SSp-ll', 'SSp-tr', 'VISa', 'VISam']],
        'rez4':  [['MOs'], ['VISam', 'VISpm'], ['RSPd']],
        'rez5':  [['SSp-m', 'SSp-n'], ['RSPd'], ['AUDp', 'TEa']],
        'rez6':  [['SSp-tr'], ['SSp-ll'], ['SSp-m', 'SSp-n', 'SSs']]
    }
}

In [ ]:
labelsCanon = list(dataDB.map_channel_labels_canon().values())

_triplet_to_string = lambda s1,s2,t: str(tuple([s1,s2,t])).replace("'", "")

for datatype, clusterDict in clusterDictDict.items():
    for tripletKey, (s1Lst, s2Lst, trgLst) in clusterDict.items():
        print(datatype, s1Lst, s2Lst, trgLst)

        dfTot = pd.DataFrame()
        groupLst = sorted(list(set(dfSummary.columns) - {'key', 'label', 'syn'}))
        for key, dataCond in dfSelectedTriplets[datatype].groupby(groupLst):
            dfSub = pd.DataFrame()
            for s1 in s1Lst:
                for s2 in s2Lst:
                    for t in trgLst:
                        dfThis = pd_query(dataCond, {'label': _triplet_to_string(s1,s2,t)})
                        if len(dfThis) > 0:
                            dfSub = dfSub.append(dfThis)

    #         print('--', key, len(dfSub))

            assert len(dfSub) > 0



            rezTmp = np.mean(dfSub['syn'])
            dfRow = pd.DataFrame(dict(zip(groupLst, key)), index=[0])
            dfRow['syn'] = rezTmp
            dfTot = dfTot.append(dfRow)

        fig, ax = plt.subplots(ncols=2, figsize=(8,4))
        plot_triplet_clusters(fig, ax[0], dataDB, s1Lst, s2Lst, trgLst)
        plot_df_2D_outer_product(ax[1], dfTot, ['mousename'],
                             ['intervName', 'trialType'],
                             'syn', vmin=0, vmax=0.06, orderDict={'intervName': ['PRE', 'TEX', 'DEL', 'REW']}, 
                             dropEmpty=True)

        plt.savefig('large_triplet_brainplot_tex_'+datatype + '_' + tripletKey + '.svg')
        plt.show()

## 2D maps

In [ ]:
mousewise.plot_2D_avg(dataDB, pwdH5data, dfSummary, 'intervName', intervNames,
                      dropChannels=[16, 26], avgAxis=2)

In [ ]:
mousewise.plot_2D_avg(dataDB, pwdH5data, dfSummary, 'trialType', trialTypes,
                      dropChannels=[16, 26], avgAxis=2)

In [ ]:
for trgChName in ['SSp-bfd', 'VISrl', 'AUDp']:
    mousewise.plot_2D_target(dataDB, pwdH5data, dfSummary, trgChName,
                         'intervName', intervNames, dropChannels=[16, 26])
    mousewise.plot_2D_target(dataDB, pwdH5data, dfSummary, trgChName,
                         'trialType', trialTypes, dropChannels=[16, 26])

In [ ]:
for trgChName in ['SSp-bfd', 'VISrl', 'AUDp']:
    mousewise.plot_2D_target_mousephase_subpre(dataDB, pwdH5data, dfSummary,
                                              trgChName, dropChannels=[16, 26])

In [ ]:
for trgChName in ['SSp-bfd', 'VISrl', 'AUDp']:
    mousewise.plot_2D_target_mousephase_submouse(dataDB, pwdH5data, dfSummary,
                                                trgChName, dropChannels=[16, 26])

In [ ]:
pid_joint.plot_2D_bytarget_synergy_cluster(dataDB, pwdAllH5_2, dfSummary, 'BC',
                                           dropChannels=None, clusterParam=0.001, dropWeakChannelThr=0.1)

In [ ]:
pid_joint.plot_unique_top_pairs(dataDB, pwdAllH5_2, dfSummary, nTop=20, dropChannels=None)

# Consistency

In [ ]:
pid_joint.plot_consistency_bymouse(pwdAllH5_2, dfSummary, dropChannels=None, kind='fisher', limits=[0, 1])

In [ ]:
pid_joint.plot_consistency_byphase(pwdAllH5_2, dfSummary, dropChannels=None,
                                   kind='fisher', limits=[0, 1], datatype='bn_trial')
pid_joint.plot_consistency_byphase(pwdAllH5_2, dfSummary, dropChannels=None, 
                                   kind='fisher', limits=[0, 1], datatype='bn_session')

In [ ]:
pid_joint.plot_consistency_bytrialtype(pwdAllH5_2, dfSummary, dropChannels=None, datatype='bn_trial',
                                       trialTypes=['Hit', 'CR'], kind='fisher', fisherThr=0.1, limits=[0, 1])
pid_joint.plot_consistency_bytrialtype(pwdAllH5_2, dfSummary, dropChannels=None, datatype='bn_session',
                                       trialTypes=['Hit', 'CR'], kind='fisher', fisherThr=0.1, limits=[0, 1])

In [ ]:
dfSummary